# Graphs of Inverse Kinematics and Dynamics from XIMU3 using ROS-OpenSIMRT 


In [ ]:
import os, sys
sys.path.append("/home/frekle/github/opensimrt/catkin_ws/tmp/refdata/refdata")
import refdata
import numpy as np
import matplotlib.pyplot as plt

import glob
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

refdata.plt.rcParams['figure.figsize'] = [12, 5]
refdata.ROW_OF_FLOTS = 1

In [ ]:
from importlib import reload
reload(refdata)

In [ ]:
from IPython.display import display, Javascript

def create_new_cell_below(content):
    display(Javascript('''
        var idx = Jupyter.notebook.get_selected_index();
        var codeCell = Jupyter.notebook.insert_cell_at_index('code', idx);
        codeCell.set_text('#GENERATED CELL!\\n%s');
        Jupyter.notebook.select(idx);
    ''' % content.replace('\n', '\\n')))

# Example usage:
new_cell_content = """
# Your commands here
print("Hello, World!")
"""

#create_new_cell_below(new_cell_content)


In [ ]:
def find_steps(x, grf):
    #refdata.plt.plot(x,grf)
    #plt.show()
    threshold = weight*9.8/10
    step = False
    step_seq = []
    step_start = 1e200
    step_stop = 1e200
    min_duration = 0.05
    lowering = False
    rising = False
    this_step = [None,None]
    for t, y in zip(x,grf):
        #print((y, threshold))
        if not step and y>threshold:
            #print(t)
            rising = True
            step_start= np.min([t,step_start])
            if t-step_start > min_duration:
                step = True
                #print("is step")
                this_step[0] = step_start
        if rising and y<threshold:
            rising = False
            step_start = 1e200
        if step and y<threshold:
            #print("trying to find lowering edge, %f, %f"%(t,step_stop))
            lowering = True
            step_stop = np.min([t,step_stop])
            if t-step_stop > min_duration:
                #print("found lowering %s"%step_stop)
                step = False
                this_step[1] = step_stop
                step_seq.append(this_step)
                this_step = [None,None]
                step_start = 1e200
                step_stop = 1e200
        if lowering and y>threshold:
            lowering = False
            step_stop = 1e200
    if this_step[0]:
        step_seq.append(this_step) ## appending the last incomplete step because we need the start for segmentation
    return step_seq

In [ ]:
def gen_step_ticks(some_steps):
    xy = [[],[]]
    for a_step in some_steps:
        xy[0].extend([a_step[0],a_step[0],a_step[0]])
        xy[1].extend([weight*9.8/10,700,np.nan])
        xy[0].extend([a_step[1],a_step[1],a_step[1]])
        xy[1].extend([weight*9.8/10,800,np.nan])
    return xy[0],xy[1]

def construct_step_segmentation_vector(some_steps):
    new_step_def_list = []
    for i,ith_step in enumerate(some_steps):
        #disregard the first step
        if i == 0:
            continue
        new_step_def_list.append(ith_step[0])
    return new_step_def_list

In [ ]:
weight= 54
conv_names = refdata.graph_params.get_id_graph_params(weight)
conv_names

In [ ]:
#2023-03-03-11-56-24walking012_ik_lower.sto

#sto_files = glob.glob("/home/frekle/github/opensimrt/catkin_ws/tmp/02/id_so_batch_walking_calcn/*.sto")
#this is overkill. I know I want to make sure I am looking at the correct file, 
#but it was getting too difficult to debug. just be extra careful!

sto_files = glob.glob("*.sto")
sto_files.sort()
sto_files

In [ ]:
ik_files = []
grfL_files = []
grfR_files = []
for i in range(5):
    for file in sto_files:
        if "SCRIPT%d"%i in file:
            if "ik.sto" in file:
                ik_files.append(file)
                continue
            if "grfLeft.sto" in file:
                grfL_files.append(file)
                continue
            if "grfRight.sto" in file:
                grfR_files.append(file)
                continue


In [ ]:
ik_files

In [ ]:
grfL_files

In [ ]:
grfR_files

In [ ]:
step_seg_l_list = []
step_seg_r_list = []

for ik_, grfL_, grfR_ in zip(ik_files, grfL_files, grfR_files):
    
    
    ik_2 = refdata.TrialData(ik_, remove_time_offset=False)
    grfL = refdata.TrialData(grfL_, remove_time_offset=False)
    grfR = refdata.TrialData(grfR_, remove_time_offset=False)
    
    zero_time = np.min(ik_2.data.time.values)
    zero_time2 = np.min(grfL.data.time.values)
    #print("\nmin ik %s"%zero_time)
    #print("\nmin insole %s"%zero_time2)
    #print((grfL.data.time-zero_time))
    x_ik = list(ik_2.data.time-zero_time)
    #print("x_ik:%s"%x_ik)
    #print("\nik_2.data.time:%s"%list(ik_2.data.time))
    
    x_grfL = list(grfL.data.time-zero_time)
    left_steps_vec = find_steps(x_grfL,grfL.data["1_ground_force_vy"])
    #for s_v in left_steps_vec:
    #    print("l: %s"%s_v)
        
    left_st_seg = construct_step_segmentation_vector(left_steps_vec)
    step_seg_l_list.append(left_st_seg)
    
    #print(left_st_seg)
    
    fig, ax1 = plt.subplots()
        
    refdata.plt.plot(x_ik,ik_2.data.ankle_angle_l/3.141592*180,"--", label="angle")
    
    xsl, ysl = gen_step_ticks(left_steps_vec)
    
    
    
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    ax1.set_ylim((-20,25))
    
    refdata.plt.plot(xsl,ysl,'g')
    
    refdata.plt.plot(x_grfL,grfL.data["1_ground_force_vy"],'r', label="y")
    refdata.plt.plot(x_grfL,grfL.data["1_ground_force_vx"],'g', label="x")
    refdata.plt.plot(x_grfL,grfL.data["1_ground_force_vz"],'b', label="z")
    refdata.plt.title("grfL")
    refdata.plt.legend()
    refdata.plt.show()
    
    fig, ax1 = plt.subplots()
    refdata.plt.plot(x_ik,ik_2.data.ankle_angle_r/3.141592*180,"--", label="angle")
    ax1.set_ylim((-20,25))
    x_grfR = grfR.data.time-zero_time
    
    right_steps_vec = find_steps(x_grfR,grfR.data["ground_force_vy"])
    #for s_v in right_steps_vec:
    #    print("r: %s"%s_v)
    
    right_st_seg = construct_step_segmentation_vector(right_steps_vec)
    step_seg_r_list.append(right_st_seg)
    
    #print(right_st_seg)
    
    xsr, ysr = gen_step_ticks(right_steps_vec)
    
    
    
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    
    refdata.plt.plot(xsr,ysr,'g')
    
    refdata.plt.plot(x_grfR,grfR.data["ground_force_vy"],'r', label="y")
    refdata.plt.plot(x_grfR,grfR.data["ground_force_vx"],'g', label="x")
    refdata.plt.plot(x_grfR,grfR.data["ground_force_vz"],'b', label="z")
    refdata.plt.title("grfR")
    refdata.plt.legend()
    refdata.plt.show()

In [ ]:
skip_trials =[]

In [ ]:
id_files = glob.glob("*tau.sto")
id_files.sort()
id_files

## Walking trials:

In [ ]:
gait_trials= []
for trial in id_files:
    for gait_name in ["gait", "walk"]:
        if gait_name in trial:
            gait_trials.append(trial)
gait_trials #= ["/home/frekle/github/opensimrt/catkin_ws/tmp/02/48_s2_id_walking_filtered_1tau.sto"]

In [ ]:
xy_knees = refdata.generate_knees_curves(gait_trials,[], curve_prefix="ankle_angle")

In [ ]:
left_str_ = "xy_time_clippings_left = {\n"
for sg_list, (name, xy_tuples) in zip(step_seg_l_list,xy_knees[0].items()):
    left_str_ +="\"%s\":%s,\n"%(name,sg_list)
left_str_ += "}\n"
right_str_ = "xy_time_clippings_right = {\n"
for sg_list, (name, xy_tuples) in zip(step_seg_r_list,xy_knees[1].items()):
    right_str_ += "\"%s\":%s,\n"%(name,sg_list)
right_str_ += "}\n"



In [ ]:
## Only run once. fix in the cell below
create_new_cell_below(left_str_)

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
for name, xy_tuples in xy_knees[0].items():
    if name == gait_trials[0]:
        #plt.title(name)        
        print(name)
        
        refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )


In [ ]:
## Only run once. fix in the cell below
create_new_cell_below(right_str_)

In [ ]:

for name, xy_tuples in xy_knees[1].items():
    if name == gait_trials[0]:
        #plt.title(name)
        print(name)
        refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )

In [ ]:
xy_clippings_both = (xy_time_clippings_left,xy_time_clippings_right)
print(xy_clippings_both[0])
print(xy_clippings_both[1])

In [ ]:
for name, xy_tuples in xy_knees[0].items():    
    for gait_name in ["gait", "walk", "fast","slow"]:
        if gait_name in name:
            print(name+r" left")        
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )
for name, xy_tuples in xy_knees[1].items():
    for gait_name in ["gait", "walk", "fast","slow"]:
        if gait_name in name:
            print(name+r" right")        
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )

In [ ]:
gait_trials

In [ ]:
skip_trials = []

In [ ]:
all_curves_for_this_person = refdata.generate_gait_plots(gait_trials, xy_clippings_both, ref=refdata.IdData(),
                                                         skip_trials=skip_trials, only_walking=True, plot_steps = True
                                                        , plot_reference=True, subplot_grid = (2,3),
                                                        conv_names=refdata.graph_params.get_id_graph_params(weight))

In [ ]:
if False:
    plt.rcParams['figure.figsize'] = [5, 5]
    for name, list_of_curves in all_curves_for_this_person.items():
        ax = plt.gca()
        ax.set_prop_cycle('color',plt.cm.inferno(np.linspace(0,1,3)))
        for curves in list_of_curves[0]:
            x = curves[0]
            for curve in curves[1]:
                plt.plot(x,curve)

        plt.show()

In [ ]:
#dir(plt.cm)
plt.rcParams['figure.dpi'] = 400 

In [ ]:
refdata.plot_std_plots(all_curves_for_this_person, plot_std=True, ref=refdata.IdData(), subplot_grid = (2,3),)

In [ ]:
refdata.plot_std_plots(all_curves_for_this_person, plot_std=False, ref=refdata.IdData(), subplot_grid = (2,3),)

In [ ]:
all_curves_for_this_person.keys()

# SO curves:

In [ ]:
so_files = glob.glob("*so.sto")
so_files.sort()
so_files

In [ ]:
so_gait_trials= []
for trial in so_files:
    for gait_name in ["gait", "walk"]:
        if gait_name in trial:
            so_gait_trials.append(trial)

In [ ]:
xy_knees_so = refdata.generate_knees_curves(so_gait_trials, skip_trials, curve_prefix="glut_med1", conv_names=refdata.graph_params.get_so_graph_params())

In [ ]:
xy_knees_list0 = []
xy_knees_list1 = []
for name, xy_tuples in xy_time_clippings_left.items():
    xy_knees_list0.append((name,xy_tuples))    
for name, xy_tuples in xy_time_clippings_right.items():
    xy_knees_list1.append((name,xy_tuples))    

In [ ]:
left_so_str = "xy_time_clippings_so_left = {\n"
for i, (name, xy_tuples) in enumerate(xy_knees_so[0].items()):
    left_so_str += "\""+name+"\":%s,\n"%str(xy_knees_list0[i][1])
left_so_str +="}\n"
right_so_str = "xy_time_clippings_so_right = {\n"
for i,(name, xy_tuples) in enumerate(xy_knees_so[1].items()):
    right_so_str +="\""+name+"\":%s,\n"%str(xy_knees_list1[i][1])
right_so_str +="}\n"

create_new_cell_below(left_so_str+right_so_str)

In [ ]:
xy_so_clippings_both = (xy_time_clippings_so_left,xy_time_clippings_so_right)
print(xy_so_clippings_both[0])
print(xy_so_clippings_both[1])

## Generates curves for all muscles

In [ ]:
all_so_curves_for_this_person = refdata.generate_gait_plots(so_gait_trials, xy_so_clippings_both, ref=None,
                                                         skip_trials=skip_trials, only_walking=True, plot_steps = True
                                                        , plot_reference=False, conv_names=refdata.graph_params.get_so_graph_params())

## Generates curves for main calf muscles

In [ ]:
selected_curves_for_this_person = refdata.generate_gait_plots(so_gait_trials, xy_so_clippings_both, ref=None,
                                                         skip_trials=skip_trials, only_walking=True, plot_steps = True
                                                        , plot_reference=False, 
                                                     conv_names=refdata.graph_params.get_so_short_graph_params(),  subplot_grid=(2, 3))

## Generates plots for every trial and mean std curves for SO

In [ ]:
refdata.plot_std_plots(selected_curves_for_this_person, plot_std=False, ref=refdata.SoData(), subplot_grid = (2,3),)

In [ ]:
plt.rcParams['figure.figsize'] = [8, 10]
refdata.plot_std_plots(selected_curves_for_this_person, plot_std=True, ref=refdata.SoData(), subplot_grid = (2,3),)

In [ ]:
refdata.SoData()

## generate graphs from id events

In [ ]:
import re

def escape_special_characters(input_string):
    # Define a dictionary of characters to be escaped
    escape_dict = {
        "&": r"\&",
        "%": r"\%",
        "$": r"\$",
        "#": r"\#",
        "_": r"\_",
        "{": r"\{",
        "}": r"\}",
        "^": r"\^{}",
        "~": r"\textasciitilde{}",
        "\\": r"\textbackslash{}"
    }
    
    # Use regular expressions to find and escape the characters
    escaped_string = re.sub(r'([&%$#_{}^~\\])', lambda x: escape_dict[x.group()], input_string)
    
    return escaped_string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [16, 12]
def eventnames(df):
    d = {}
    for coln in df.columns:
        if "field.header.stamp" == coln:
            #real_name = df["field.header.frame_id"]
            real_name = "ik produced"
            d.update({coln: real_name})
        if "stamp" in coln and "event" in coln:
            real_name_col = ".".join(coln.split(".")[0:3])+".name"
            real_name = df[real_name_col][0]
            d.update ({coln: real_name})
    #for coln, real_name in d.items():
    #    print("%s -> %s"%(coln, real_name))
    return d
#wanted_cols = eventnames(df)

def get_cols(df):
    wanted_cols = eventnames(df)
    a = []
    for v in wanted_cols:
        a.append(v)
        #print(v)
    return a, wanted_cols

def rename_df(df):
    a = get_cols(df)
    return df[a[0]].rename(columns=a[1])

def plot_differences(df):
    
    fig, ax = plt.subplots()
    for i, col in enumerate(df.columns):
        if i == 0:
            continue
        this_column = df.iloc[:,i]
        #print(this_column)
        ref_column = df.iloc[:,i-1]
        #print(ref_column)
        
        y = this_column -ref_column
        y_label = df.columns[i] +" - " + df.columns[i-1]
        print("%s:\t %f ms"%(y_label+" mean", np.mean(y)/1000000))
        y_label_cleaned = escape_special_characters(y_label)
        ax.plot(y, label= y_label_cleaned)
    ax.legend()
    
for i in range(5):
    df = pd.read_csv("id_output_walking_%d.bag_timings.txt"%i)
    a = rename_df(df)
    plot_differences(a)
    plt.show()

# Augmented Reality part

In [ ]:
from scipy.signal import savgol_filter
from copy import deepcopy
def filter_stuff(curves_for_person_):
    curves_for_person = deepcopy(curves_for_person_)
    for joint in curves_for_person.keys():
        for num_curve in range(len(curves_for_person[joint][0][0][1])):
            curves_for_person[joint][0][0][1][num_curve] = savgol_filter(curves_for_person[joint][0][0][1][num_curve],10,3)
    return curves_for_person

filtered_curved = filter_stuff(all_curves_for_this_person)


In [ ]:
refdata.plot_std_plots(filtered_curved, plot_std=False, ref=refdata.IdData(), subplot_grid = (2,3),)

In [ ]:
from scipy.signal import savgol_filter
x= all_curves_for_this_person['hip_flexion_l'][0][0][0]
y= all_curves_for_this_person['hip_flexion_l'][0][0][1][0]
yhat = savgol_filter(y, 21, 3) # window size 51, polynomial order 3

plt.plot(x,y)
plt.plot(x,yhat, color='red')
plt.show()


In [ ]:
refdata.plot_std_plots(all_curves_for_this_person, ref=refdata.IdData(), plot_std=False, subplot_grid = (2,3),)

# Saving figures part

This is old, needs to be updated before being used!

In [ ]:
import pickle


In [ ]:
subject_data_filename_prefix = os.getcwd().split("experiment_data/")[1].split("/")[0]

In [ ]:
subject_data_filename_prefix

In [ ]:
subject_data_directory = os.getcwd().split("experiment_data/")[0]+"experiment_data/"

In [ ]:
subject_complete_filename = "{}{}_allcurves.pkl".format(subject_data_directory,subject_data_filename_prefix)
print(subject_complete_filename)
f = open(subject_complete_filename,"wb")
pickle.dump(all_curves_for_this_person,f)
f.close()

In [ ]:
import pandas as pd
import matplotlib
matplotlib.rc('text', usetex = False)
plt.rcParams['figure.figsize'] = [18, 5]
for file in ik_files[2:]:
    print("file: %s"%file)
    data = pd.read_csv (file, sep = '\t', skiprows=4)

    #remove time ofset
    data.time = data.time - data.time[0]

    data.set_index('time')
    print(data.columns)

    data.plot(x="time", y=["lumbar_extension","lumbar_bending","lumbar_rotation"], ylim=(-1.57,1.57))

    plt.tight_layout()
    plt.show()
    
    data.plot(x="time", y=["pelvis_tilt","pelvis_list","pelvis_rotation"], ylim=(-1.57,1.57))

    plt.tight_layout()
    plt.show()

    data.plot(x="time", y=["hip_flexion_r","hip_adduction_r","hip_rotation_r"], ylim=(-1.57,1.57))
    plt.tight_layout()
    plt.show()

    data.plot(x="time", y=["hip_flexion_l","hip_adduction_l","hip_rotation_l"], ylim=(-1.57,1.57))
    plt.tight_layout()
    plt.show()

    data.plot(x="time", y=["knee_angle_r","knee_angle_l"], ylim=(-1.57,1.57))
    plt.tight_layout()
    plt.show()

    data.plot(x="time", y=["ankle_angle_r","ankle_angle_l"], ylim=(-1.57,1.57))
    plt.tight_layout()
    plt.show()